<a href="https://colab.research.google.com/github/gabrielcardoso21/boletim-de-urna/blob/main/Visualize_votes_in_S%C3%A3o_Paulo_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'locais-de-votacao-sp:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2538652%2F4309710%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240727%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240727T045030Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5d5a7f9d4ff165fa85665d121b58a3f18f54f449764ff22806e9a1688db730c2843365b7157f86db8a4fc53d56bd6cfa5421345ba91f3733abc5cc12dc4db7e36da2111c3b50a8e8a021a6409a2c59e57ef0125b2107492eafa001ac18ac2582fca3e26c3b8e9f0a3436020eeed29922ab43da4669ff86e0d3fb6c0a729f900246d3baa66cbb1ed4b9b6fe0e407e25f0f5ee370ec3908c165df4b8d9d68738d97131c3566c2bb81ac76b083cab9e1e0221635fb132b0e73d239c452f5ce811e86947c4e9152d5ed0b075f5f521e1936b700561dc6ae615f3e34d7811dab78fd44f735e610b22e9780624b9d6bbe0e0ff41c2990a60b940d376ea443d67804173,boletins-de-urna-do-estado-de-so-paulo-2022:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5449625%2F9039721%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240727%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240727T045030Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D907de0ee46a7c2f5356e4c53f639314988ee8732b2a316e10a945f82a7ac7000ec292127d4a518c2b2aa76a7e604058745f5076a245bbe787e3990cca72e4530c4a4dd622fd438fb5c367fafeba06f71caa0c37c402b7291221e87713e81af83b8d429422fbbb0e1513b6e002f6a44a2c2f65485f9f79ddac2ce148532013af3c7b024ea5f55bb2f62a208d88ed5365873ed59aca691cfd826fe87794a3b04a7c41c7a014fbde29e5b5cff0d49af2c6a8dcfd9dee8d24418cc3734c1f5176eee6660dc01ddd791b1a087db090a3c09c77f490ea33379c5a42ca961a4ea580c382af35c95f8c5d113f33d8893c92b9bad22dab75137519dfceee4445e93d19f61'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 506504 bytes downloaded
Downloaded and uncompressed: locais-de-votacao-sp
[==================================================] 444518291 bytes downloaded
Downloaded and uncompressed: boletins-de-urna-do-estado-de-so-paulo-2022
Data source import complete.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/boletins-de-urna-do-estado-de-so-paulo-2022/bweb_1t_SP_051020221321/bweb_1t_SP_051020221321.csv
/kaggle/input/boletins-de-urna-do-estado-de-so-paulo-2022/bweb_1t_SP_051020221321/leiame-boletimurnaweb.pdf
/kaggle/input/boletins-de-urna-do-estado-de-so-paulo-2022/bweb_2t_SP_311020221535/leiame-boletimurnaweb.pdf
/kaggle/input/boletins-de-urna-do-estado-de-so-paulo-2022/bweb_2t_SP_311020221535/bweb_2t_SP_311020221535.csv
/kaggle/input/locais-de-votacao-sp/arquivo.csv


# **Project Idea:**

The project aims to conduct a detailed analysis and provide a comprehensive overview of the voting data from the first and second rounds of the 2022 elections. By merging datasets containing the election results from each round with the names of the schools where the elections were held, the objective is to aggregate and present the voting data from the broadest to the most specific levels. This includes summarizing the total votes by state, city, neighborhood, and school. The analysis will provide valuable insights into voting patterns and behaviors across different regions and demographics.

# **Justification:**

The analysis of electoral data is crucial for various social entities that seek to better understand the democratic process and the factors influencing election outcomes. Political parties, non-governmental organizations (NGOs), political analysts, and academic institutions can significantly benefit from the insights derived from this analysis. For political parties, understanding voting patterns can aid in planning more effective campaign strategies. NGOs and institutions that promote electoral transparency and integrity can use the data to identify areas of concern and improve civic education. Political analysts and academics can explore the data to develop theories and studies that contribute to the strengthening of democracy. Additionally, the media can use these insights to better inform the public about electoral behavior and its implications. Therefore, this project not only enhances the understanding of election results but also promotes transparency and informed participation in the democratic process.

# Detailed Procedure
## **Phase 1: Data Cleaning**

The first phase consists of cleaning the datasets of the voting records to retain only relevant information. Below is the step-by-step procedure executed so far:

### 1. Load Data from the First Round

In [ ]:
b1t = pd.read_csv('/kaggle/input/boletins-de-urna-do-estado-de-so-paulo-2022/bweb_1t_SP_051020221321/bweb_1t_SP_051020221321.csv', encoding='latin-1', sep=';', nrows=10000)
b1t.head(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_PLEITO,DT_PLEITO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,...,CD_FLASHCARD_URNA_EFETIVADA,DT_CARGA_URNA_EFETIVADA,DS_CARGO_PERGUNTA_SECAO,DS_AGREGADAS,DT_ABERTURA,DT_ENCERRAMENTO,QT_ELEITORES_BIOMETRIA_NH,DT_EMISSAO_BU,NR_JUNTA_APURADORA,NR_TURMA_APURADORA
0,05/10/2022,16:02:45,2022,0,Eleição Ordinária,406,02/10/2022,1,544,Eleição Geral Federal 2022,...,A836CFA2,22/09/2022 11:04:00,1 - 1,#NULO#,02/10/2022 08:00:01,02/10/2022 17:10:40,52,02/10/2022 17:12:40,-1,-1
1,05/10/2022,16:02:45,2022,0,Eleição Ordinária,406,02/10/2022,1,544,Eleição Geral Federal 2022,...,A836CFA2,22/09/2022 11:04:00,1 - 1,#NULO#,02/10/2022 08:00:01,02/10/2022 17:10:40,52,02/10/2022 17:12:40,-1,-1
2,05/10/2022,16:02:45,2022,0,Eleição Ordinária,406,02/10/2022,1,544,Eleição Geral Federal 2022,...,A836CFA2,22/09/2022 11:04:00,1 - 1,#NULO#,02/10/2022 08:00:01,02/10/2022 17:10:40,52,02/10/2022 17:12:40,-1,-1
3,05/10/2022,16:02:45,2022,0,Eleição Ordinária,406,02/10/2022,1,544,Eleição Geral Federal 2022,...,A836CFA2,22/09/2022 11:04:00,1 - 1,#NULO#,02/10/2022 08:00:01,02/10/2022 17:10:40,52,02/10/2022 17:12:40,-1,-1
4,05/10/2022,16:02:45,2022,0,Eleição Ordinária,406,02/10/2022,1,544,Eleição Geral Federal 2022,...,A836CFA2,22/09/2022 11:04:00,1 - 1,#NULO#,02/10/2022 08:00:01,02/10/2022 17:10:40,52,02/10/2022 17:12:40,-1,-1


### 2. Filter Relevant Features

In [ ]:
b1t.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_PLEITO', 'DT_PLEITO', 'NR_TURNO', 'CD_ELEICAO',
       'DS_ELEICAO', 'SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA',
       'NR_SECAO', 'NR_LOCAL_VOTACAO', 'CD_CARGO_PERGUNTA',
       'DS_CARGO_PERGUNTA', 'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO',
       'DT_BU_RECEBIDO', 'QT_APTOS', 'QT_COMPARECIMENTO', 'QT_ABSTENCOES',
       'CD_TIPO_URNA', 'DS_TIPO_URNA', 'CD_TIPO_VOTAVEL', 'DS_TIPO_VOTAVEL',
       'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS', 'NR_URNA_EFETIVADA',
       'CD_CARGA_1_URNA_EFETIVADA', 'CD_CARGA_2_URNA_EFETIVADA',
       'CD_FLASHCARD_URNA_EFETIVADA', 'DT_CARGA_URNA_EFETIVADA',
       'DS_CARGO_PERGUNTA_SECAO', 'DS_AGREGADAS', 'DT_ABERTURA',
       'DT_ENCERRAMENTO', 'QT_ELEITORES_BIOMETRIA_NH', 'DT_EMISSAO_BU',
       'NR_JUNTA_APURADORA', 'NR_TURMA_APURADORA'],
      dtype='object')

In [ ]:
features = ['CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO', 'CD_CARGO_PERGUNTA', 'DS_CARGO_PERGUNTA', 'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'QT_APTOS', 'QT_COMPARECIMENTO', 'QT_ABSTENCOES', 'CD_TIPO_URNA', 'DS_TIPO_URNA', 'CD_TIPO_VOTAVEL', 'DS_TIPO_VOTAVEL', 'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS', 'NR_URNA_EFETIVADA', 'DS_CARGO_PERGUNTA_SECAO', 'DS_AGREGADAS', 'QT_ELEITORES_BIOMETRIA_NH', 'NR_JUNTA_APURADORA', 'NR_TURMA_APURADORA']

In [ ]:
b1t[features].head(10)

,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,...,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA,DS_CARGO_PERGUNTA_SECAO,DS_AGREGADAS,QT_ELEITORES_BIOMETRIA_NH,NR_JUNTA_APURADORA,NR_TURMA_APURADORA
0,71072,SÃO PAULO,1,1,1015,1,Presidente,12,PDT,Partido Democrático Trabalhista,...,Nominal,12,CIRO GOMES,13,1825364,1 - 1,#NULO#,52,-1,-1
1,71072,SÃO PAULO,1,1,1015,1,Presidente,15,MDB,Movimento Democrático Brasileiro,...,Nominal,15,SIMONE TEBET,41,1825364,1 - 1,#NULO#,52,-1,-1
2,71072,SÃO PAULO,1,1,1015,1,Presidente,22,PL,Partido Liberal,...,Nominal,22,JAIR BOLSONARO,113,1825364,1 - 1,#NULO#,52,-1,-1
3,71072,SÃO PAULO,1,1,1015,1,Presidente,13,PT,Partido dos Trabalhadores,...,Nominal,13,LULA,98,1825364,1 - 1,#NULO#,52,-1,-1
4,71072,SÃO PAULO,1,1,1015,1,Presidente,44,UNIÃO,UNIÃO BRASIL,...,Nominal,44,SORAYA THRONICKE,1,1825364,1 - 1,#NULO#,52,-1,-1
5,71072,SÃO PAULO,1,1,1015,1,Presidente,30,NOVO,Partido Novo,...,Nominal,30,FELIPE D'AVILA,5,1825364,1 - 1,#NULO#,52,-1,-1
6,71072,SÃO PAULO,1,1,1015,1,Presidente,-1,#NULO#,#NULO#,...,Nulo,96,Nulo,4,1825364,1 - 1,#NULO#,52,-1,-1
7,71072,SÃO PAULO,1,1,1015,1,Presidente,-1,#NULO#,#NULO#,...,Branco,95,Branco,4,1825364,1 - 1,#NULO#,52,-1,-1
8,71072,SÃO PAULO,1,1,1015,3,Governador,-1,#NULO#,#NULO#,...,Branco,95,Branco,5,1825364,3 - 1,#NULO#,52,-1,-1
9,71072,SÃO PAULO,1,1,1015,3,Governador,45,PSDB,Partido da Social Democracia Brasileira,...,Nominal,45,RODRIGO GARCIA,72,1825364,3 - 1,#NULO#,52,-1,-1


### 3. Remove Duplicate Information

There are still too many columns. This is because many of them are duplicate information. If we have the code, we don't need the name. Let's reduce some of the duplicate columns. And to garantee we know what we're doing, we're going to separate features for visualize the data and features to work with (numerical codes).

In [ ]:
features_visualize = ['NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO', 'DS_CARGO_PERGUNTA', 'SG_PARTIDO', 'QT_APTOS', 'QT_COMPARECIMENTO', 'QT_ABSTENCOES', 'DS_TIPO_URNA', 'DS_TIPO_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS', 'NR_URNA_EFETIVADA', 'DS_CARGO_PERGUNTA_SECAO', 'DS_AGREGADAS', 'QT_ELEITORES_BIOMETRIA_NH', 'NR_JUNTA_APURADORA', 'NR_TURMA_APURADORA']
features_useful = ['CD_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO', 'CD_CARGO_PERGUNTA', 'NR_PARTIDO', 'QT_APTOS', 'QT_COMPARECIMENTO', 'QT_ABSTENCOES', 'CD_TIPO_URNA', 'CD_TIPO_VOTAVEL', 'NR_VOTAVEL', 'QT_VOTOS', 'NR_URNA_EFETIVADA', 'DS_CARGO_PERGUNTA_SECAO', 'DS_AGREGADAS', 'QT_ELEITORES_BIOMETRIA_NH', 'NR_JUNTA_APURADORA', 'NR_TURMA_APURADORA']

In [ ]:
b1t[features_visualize].head(5)

,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,DS_CARGO_PERGUNTA,SG_PARTIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,DS_TIPO_URNA,DS_TIPO_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA,DS_CARGO_PERGUNTA_SECAO,DS_AGREGADAS,QT_ELEITORES_BIOMETRIA_NH,NR_JUNTA_APURADORA,NR_TURMA_APURADORA
0,SÃO PAULO,1,1,1015,Presidente,PDT,370,279,91,APURADA,Nominal,CIRO GOMES,13,1825364,1 - 1,#NULO#,52,-1,-1
1,SÃO PAULO,1,1,1015,Presidente,MDB,370,279,91,APURADA,Nominal,SIMONE TEBET,41,1825364,1 - 1,#NULO#,52,-1,-1
2,SÃO PAULO,1,1,1015,Presidente,PL,370,279,91,APURADA,Nominal,JAIR BOLSONARO,113,1825364,1 - 1,#NULO#,52,-1,-1
3,SÃO PAULO,1,1,1015,Presidente,PT,370,279,91,APURADA,Nominal,LULA,98,1825364,1 - 1,#NULO#,52,-1,-1
4,SÃO PAULO,1,1,1015,Presidente,UNIÃO,370,279,91,APURADA,Nominal,SORAYA THRONICKE,1,1825364,1 - 1,#NULO#,52,-1,-1


### 4. Remove Non-Useful Features

Now we can see that many features are holding the same information but with another code. Let's remove the duplicates.

In [ ]:
features_visualize.remove('DS_CARGO_PERGUNTA_SECAO')
features_useful.remove('DS_CARGO_PERGUNTA_SECAO')

Now, we can notice that some of these informations are not useful for predicting the results in the second round of the elections. Let's remove them.

In [ ]:
features_visualize.remove('DS_AGREGADAS')
features_visualize.remove('QT_ELEITORES_BIOMETRIA_NH')
features_visualize.remove('NR_JUNTA_APURADORA')
features_visualize.remove('NR_TURMA_APURADORA')
features_useful.remove('DS_AGREGADAS')
features_useful.remove('QT_ELEITORES_BIOMETRIA_NH')
features_useful.remove('NR_JUNTA_APURADORA')
features_useful.remove('NR_TURMA_APURADORA')

In [ ]:
b1t[features_visualize].head(5)

,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,DS_CARGO_PERGUNTA,SG_PARTIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,DS_TIPO_URNA,DS_TIPO_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA
0,SÃO PAULO,1,1,1015,Presidente,PDT,370,279,91,APURADA,Nominal,CIRO GOMES,13,1825364
1,SÃO PAULO,1,1,1015,Presidente,MDB,370,279,91,APURADA,Nominal,SIMONE TEBET,41,1825364
2,SÃO PAULO,1,1,1015,Presidente,PL,370,279,91,APURADA,Nominal,JAIR BOLSONARO,113,1825364
3,SÃO PAULO,1,1,1015,Presidente,PT,370,279,91,APURADA,Nominal,LULA,98,1825364
4,SÃO PAULO,1,1,1015,Presidente,UNIÃO,370,279,91,APURADA,Nominal,SORAYA THRONICKE,1,1825364


In [ ]:
features_useful

['CD_MUNICIPIO',
 'NR_ZONA',
 'NR_SECAO',
 'NR_LOCAL_VOTACAO',
 'CD_CARGO_PERGUNTA',
 'NR_PARTIDO',
 'QT_APTOS',
 'QT_COMPARECIMENTO',
 'QT_ABSTENCOES',
 'CD_TIPO_URNA',
 'CD_TIPO_VOTAVEL',
 'NR_VOTAVEL',
 'QT_VOTOS',
 'NR_URNA_EFETIVADA']

### 5. Remove Redundant Features

Now, I firmly believe that all urns are equal before the law. So, informations about the urn are not going to influence the final results.

In [ ]:
features_visualize.remove('DS_TIPO_URNA')
features_useful.remove('CD_TIPO_URNA')

In [ ]:
b1t[features_visualize].head(5)

,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,DS_CARGO_PERGUNTA,SG_PARTIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,DS_TIPO_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA
0,SÃO PAULO,1,1,1015,Presidente,PDT,370,279,91,Nominal,CIRO GOMES,13,1825364
1,SÃO PAULO,1,1,1015,Presidente,MDB,370,279,91,Nominal,SIMONE TEBET,41,1825364
2,SÃO PAULO,1,1,1015,Presidente,PL,370,279,91,Nominal,JAIR BOLSONARO,113,1825364
3,SÃO PAULO,1,1,1015,Presidente,PT,370,279,91,Nominal,LULA,98,1825364
4,SÃO PAULO,1,1,1015,Presidente,UNIÃO,370,279,91,Nominal,SORAYA THRONICKE,1,1825364


### 6. Retain Essential Information

Now let's dive in a little bit further. The features QT_APTOS, QT_COMPARECIMENTO and QT_ABSTENCOES are telling us the same information. We could drop any of them and the information would still be there. I'll randomly choose the QT_COMPARECIMENTO feature.

In [ ]:
features_visualize.remove('QT_COMPARECIMENTO')
features_useful.remove('QT_COMPARECIMENTO')

In [ ]:
b1t[features_visualize].head(10)

,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,DS_CARGO_PERGUNTA,SG_PARTIDO,QT_APTOS,QT_ABSTENCOES,DS_TIPO_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA
0,SÃO PAULO,1,1,1015,Presidente,PDT,370,91,Nominal,CIRO GOMES,13,1825364
1,SÃO PAULO,1,1,1015,Presidente,MDB,370,91,Nominal,SIMONE TEBET,41,1825364
2,SÃO PAULO,1,1,1015,Presidente,PL,370,91,Nominal,JAIR BOLSONARO,113,1825364
3,SÃO PAULO,1,1,1015,Presidente,PT,370,91,Nominal,LULA,98,1825364
4,SÃO PAULO,1,1,1015,Presidente,UNIÃO,370,91,Nominal,SORAYA THRONICKE,1,1825364
5,SÃO PAULO,1,1,1015,Presidente,NOVO,370,91,Nominal,FELIPE D'AVILA,5,1825364
6,SÃO PAULO,1,1,1015,Presidente,#NULO#,370,91,Nulo,Nulo,4,1825364
7,SÃO PAULO,1,1,1015,Presidente,#NULO#,370,91,Branco,Branco,4,1825364
8,SÃO PAULO,1,1,1015,Governador,#NULO#,370,91,Branco,Branco,5,1825364
9,SÃO PAULO,1,1,1015,Governador,PSDB,370,91,Nominal,RODRIGO GARCIA,72,1825364


## Phase 2: Memory Optimization

The second phase involves reducing the memory occupied by the dataset.

In [ ]:
features_visualize

['NM_MUNICIPIO',
 'NR_ZONA',
 'NR_SECAO',
 'NR_LOCAL_VOTACAO',
 'DS_CARGO_PERGUNTA',
 'SG_PARTIDO',
 'QT_APTOS',
 'QT_ABSTENCOES',
 'DS_TIPO_VOTAVEL',
 'NM_VOTAVEL',
 'QT_VOTOS',
 'NR_URNA_EFETIVADA']

In [ ]:
final_features = list(set(features_visualize + features_useful))

b1t = b1t[final_features].copy()

b1t.head(5)

,NR_URNA_EFETIVADA,NR_SECAO,NM_MUNICIPIO,NR_LOCAL_VOTACAO,CD_MUNICIPIO,QT_ABSTENCOES,DS_CARGO_PERGUNTA,NM_VOTAVEL,NR_PARTIDO,CD_TIPO_VOTAVEL,NR_VOTAVEL,NR_ZONA,QT_APTOS,CD_CARGO_PERGUNTA,SG_PARTIDO,DS_TIPO_VOTAVEL,QT_VOTOS
0,1825364,1,SÃO PAULO,1015,71072,91,Presidente,CIRO GOMES,12,1,12,1,370,1,PDT,Nominal,13
1,1825364,1,SÃO PAULO,1015,71072,91,Presidente,SIMONE TEBET,15,1,15,1,370,1,MDB,Nominal,41
2,1825364,1,SÃO PAULO,1015,71072,91,Presidente,JAIR BOLSONARO,22,1,22,1,370,1,PL,Nominal,113
3,1825364,1,SÃO PAULO,1015,71072,91,Presidente,LULA,13,1,13,1,370,1,PT,Nominal,98
4,1825364,1,SÃO PAULO,1015,71072,91,Presidente,SORAYA THRONICKE,44,1,44,1,370,1,UNIÃO,Nominal,1


### 1. Optimize Integer Columns

In [ ]:
int_columns = list(b1t.select_dtypes("int64").columns)
b1t[int_columns] = b1t[int_columns].astype("uint8")

### 2. Optimize Float Columns

In [ ]:
float_columns = b1t.select_dtypes("float64").columns
b1t[float_columns] = b1t[float_columns].astype("float16")

### 3. Optimize Date Columns

In [ ]:
date_columns = [col for col in b1t.select_dtypes("object").columns if col.startswith('DT_')]
b1t[date_columns] = b1t[date_columns].astype("str")

### 4. Optimize Categorical Columns

In [ ]:
cat_columns = [col for col in b1t.select_dtypes("object").columns if not col.startswith('DT_')]
b1t[cat_columns] = b1t[cat_columns].astype("category")

### 5. Save the dtypes

In [ ]:
dtypes = dict(zip(b1t.columns.values, [b1t[col].dtype.name for col in b1t.columns]))

## Phase 3: Merge databases

### 1. Load the full data from b1t

In [ ]:
b1t = pd.read_csv('/kaggle/input/boletins-de-urna-do-estado-de-so-paulo-2022/bweb_1t_SP_051020221321/bweb_1t_SP_051020221321.csv', encoding='latin-1', sep=';', dtype=dtypes, usecols=final_features)

In [ ]:
b1t.shape

(19167373, 17)

### 2. Load the polling place data

Let's load the other database, that will give us the possibility of knowing from which neighborhood our votes came from.

In [ ]:
headers = ['Zona', 'Escola', 'Endereço', 'Cidade', 'Seções', 'Seções Especiais']

polling_place = pd.read_csv('/kaggle/input/locais-de-votacao-sp/arquivo.csv', encoding='latin-1', usecols=[0, 1, 2, 3, 4, 5], names=headers, header=None)

### 3. Compare the 2 databases

Let's analyse the cities from which our votes came from.

In [ ]:
cities_to_replace = set(sorted(b1t['NM_MUNICIPIO'].unique())) - set(sorted(polling_place.Cidade.unique()))
cities_to_replace

{'BERNARDINO DE CAMPOS',
 'BIRIGUI',
 'EMBU DAS ARTES',
 'GUAIMBÊ',
 'ICÉM',
 'ORINDIÚVA',
 'RUBIÁCEA',
 'SÃO JOÃO DE IRACEMA',
 'TAIÚVA',
 'TEJUPÁ',
 'VALPARAÍSO'}

In [ ]:
cities_to_fix = set(sorted(polling_place.Cidade.unique())) - set(sorted(b1t['NM_MUNICIPIO'].unique()))
cities_to_fix

{'BERNADINO DE CAMPOS',
 'BIRIGÜI',
 'EMBU',
 'GUAIMBÉ',
 'ICEM',
 'ORINDIUVA',
 'RUBIACÉA',
 'SAO JOÃO DE IRACEMA',
 'TAIUVA',
 'TEJUPA',
 'VALPARAÍZO'}

As the ballot box is our true source of information, we'll replace the names in the polling place to the names in the ballot box.

In [ ]:
polling_place.replace(cities_to_replace, cities_to_fix, inplace=True)

### 4. Reduce resources consumption

To consume less resources, we'll split the ballot box in different cities.

In [ ]:
!pip install unidecode
!rm -rf cities_1t
!mkdir cities_1t

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.8 MB/s eta 0:00:00


We need to treat the data from the polling place to match the data in the ballot box. We'll also attach two new columns to our ballot box: 'Escola' and 'Endereço'.

In [ ]:
from unidecode import unidecode
from IPython.display import clear_output
import os
import re

def get_city_file_name(city, include_extension=True):
    name_file = unidecode(city.lower()) + ('.csv' if include_extension else '')
    file_name = name_file.replace(' ', '_').replace("'", "_")
    return file_name

def complement_ballot_box(bb):
    ZONES = list(bb.NR_ZONA.unique())
    places_for_zones = []
    for zone in ZONES:
        zone_places = polling_place.loc[polling_place['Zona'].str.startswith(str(zone) + 'a.')]
        places_for_zones.append(zone_places)
    places = pd.concat(places_for_zones)

    def get_list_of_sections(row):
        tracks = row.split(';')
        list_of_sections = []
        for track in tracks:
            track = track.replace(' ', '')
            if track.startswith('Da'):
                first, last = [int(re.sub("[^0-9]", "", num)) for num in track.split('à')]
                list_of_sections += [i for i in range(first, last+1)]
            elif re.sub("[^0-9]", "", track):
                list_of_sections.append(int(re.sub("[^0-9]", "", track)))
        return list_of_sections

    places['Seções em lista'] = places['Seções'].map(get_list_of_sections)

    escola_dict = {}
    for index, row in places.iterrows():
        for secao in row['Seções em lista']:
            escola_dict[int(secao)] = (row['Escola'], row['Endereço'])

    def get_Escola_Endereco(section):
        return escola_dict.get(int(section), ('NOT FOUND', 'NOT FOUND'))

    escola_endereco = bb['NR_SECAO'].map(get_Escola_Endereco)
    bb['Escola'] = escola_endereco.apply(lambda x: x[0])
    bb['Endereço'] = escola_endereco.apply(lambda x: x[1])
    return bb

cities = sorted(b1t['NM_MUNICIPIO'].unique())
for ind, city in enumerate(cities):
    if os.path.isfile("cities_1t/" + get_city_file_name(city, False) + '.csv'):
        continue
    clear_output(True)
    print(city, ' - ', ind, '/', len(cities))
    b1t_city = b1t.loc[b1t.NM_MUNICIPIO == city]
    b1t_city = complement_ballot_box(b1t_city.copy())
    b1t_city.to_csv("cities_1t/" + get_city_file_name(city), sep=';', encoding='latin-1')

ÓLEO  -  644 / 645


In [ ]:
positions = sorted(b1t['DS_CARGO_PERGUNTA'].unique())
schools = sorted(polling_place.Escola.unique())

Now we don't need b1t anymore.

In [ ]:
del b1t

### 5. Load the second round

Let's load the second round file and apply the same modifications to it.

In [ ]:
b2t = pd.read_csv('/kaggle/input/boletins-de-urna-do-estado-de-so-paulo-2022/bweb_2t_SP_311020221535/bweb_2t_SP_311020221535.csv', encoding='latin-1', sep=';', dtype=dtypes, usecols=final_features)

In [ ]:
!rm -rf cities_2t
!mkdir cities_2t

In [ ]:
from unidecode import unidecode
from IPython.display import clear_output
import os

cities = sorted(b2t['NM_MUNICIPIO'].unique())
for ind, city in enumerate(cities):
    if os.path.isfile("cities_2t/" + get_city_file_name(city, False) + '.csv'):
        continue
    clear_output(True)
    print(city, ' - ', ind, '/', len(cities))
    bb_city = b2t.loc[b2t.NM_MUNICIPIO == city]
    bb_city = complement_ballot_box(bb_city.copy())
    bb_city.to_csv("cities_2t/" + get_city_file_name(city), sep=';', encoding='latin-1')

ÓLEO  -  644 / 645


In [ ]:
!pip install gradio --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
final_features += ['Escola', 'Endereço']

In [ ]:
import pandas as pd
from unidecode import unidecode
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import lru_cache

turn = 1
school = False
position = False
voting_bulletin = False

# Function to get the file name of the municipality
def get_city_file_name(city, extension='csv'):
    name_file = unidecode(city.lower()) + '.' + extension
    file_name = name_file.replace(' ', '_').replace("'", "_")
    return file_name

# Function to read the city file with caching
@lru_cache(maxsize=None)
def read_city_file(city):
    turn = globals().get('turn', 1)
    voting_bulletin = pd.read_csv(f'cities_{turn}t/' + get_city_file_name(city), encoding='latin-1', sep=';', dtype=dtypes, usecols=final_features)
    return voting_bulletin

# Function to set the municipality
def set_city(city):
    globals()['city'] = city
    voting_bulletin = read_city_file(city)
    globals()['voting_bulletin'] = voting_bulletin
    return gr.update(choices=list(voting_bulletin['Escola'].unique()), value=None, interactive=True), voting_bulletin.head()

# Function to set the position
def set_position(position):
    voting_bulletin = globals().get('voting_bulletin', False)
    if position:
        voting_bulletin = voting_bulletin[voting_bulletin['DS_CARGO_PERGUNTA'] == position]
    globals()['position'] = position
    globals()['voting_bulletin'] = voting_bulletin
    return voting_bulletin.head()

# Function to set the school
def set_school(school):
    voting_bulletin = globals().get('voting_bulletin', False)
    if school:
        voting_bulletin = voting_bulletin[voting_bulletin['Escola'] == school]
    globals()['school'] = school
    globals()['voting_bulletin'] = voting_bulletin
    return voting_bulletin.head()

def set_turn(turn):
    globals()['turn'] = turn
    city = globals().get('city', False)
    school = globals().get('school', False)
    position = globals().get('position', False)
    if city:
        set_city(city)
    if school:
        set_school(school)
    if position:
        set_position(position)
    voting_bulletin = globals()['voting_bulletin']
    return voting_bulletin.head()

# Function to plot the selected type
def plot_selected_type():
    voting_bulletin = globals().get('voting_bulletin', pd.DataFrame())
    result = voting_bulletin.copy()

    result['NM_VOTAVEL'] = result['NM_VOTAVEL'].astype("str")
    result = result.groupby('NM_VOTAVEL')['QT_VOTOS'].sum().reset_index()

    fig, ax = plt.subplots(figsize=(16, 6))
    sns.barplot(x='NM_VOTAVEL', y='QT_VOTOS', data=result, ax=ax)
    plt.title(f'Results for {school}')
    return fig

# Gradio interface
with gr.Blocks() as demo:
    turn_dropdown = gr.Dropdown(choices=[1, 2], label="Turn")
    city_dropdown = gr.Dropdown(choices=cities, label="City")
    position_dropdown = gr.Dropdown(choices=positions, label="Position")
    school_dropdown = gr.Dropdown(choices=schools, label="School")
    output_table = gr.DataFrame(label="Results")
    output_plot = gr.Plot(label="Vote Distribution")

    turn_dropdown.change(set_turn, inputs=turn_dropdown, outputs=output_table)
    city_dropdown.change(set_city, inputs=city_dropdown, outputs=[school_dropdown, output_table])
    position_dropdown.change(set_position, inputs=position_dropdown, outputs=output_table)
    school_dropdown.input(set_school, inputs=school_dropdown, outputs=output_table)
    output_table.change(plot_selected_type, inputs=None, outputs=output_plot)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7388033db182873aca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
